In [1]:
import random
import copy
import numpy as np
import theano as T
import theano.tensor as tt
import pymc3 as pm
import pandas as pd
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
%load_ext rpy2.ipython




/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

%%R

library(ggplot2)
library(reshape)
library(grid)
library(dplyr)
library(gridExtra)
library(lme4)


paper_theme_2 <- theme_light() + theme(axis.title.x = element_text(size=18),
  axis.text.x=element_text(colour="black", 
                           size = 14), 
  axis.title.y = element_text(size = 18, vjust = 1),
  axis.text.y  = element_text(size = 14, colour="black"),
 title =element_text(size=16, face='bold'))



data_resp_loc <- read.csv("data/discrimination_dot_gaze.csv", sep="\t")
data_tracker <- read.csv("data/discrimination_tracker_data.csv", sep="\t")
data_tracker$trial_id <- floor((data_tracker$trial_id - 1)/2)
data_resp_loc$trial_id <- data_resp_loc$trial_id - 1


data_resp_loc <- data_resp_loc %>%
                filter(belowX >= 0) #%>%
                #mutate(cond=paste(round(Time1),round(Time2),sep="_"))

trials <- (unique(data_resp_loc$trial_id))
data_tracker <- data_tracker %>%
                group_by(trial_id) %>%
                filter(trial_id %in% trials)

data_resp_loc$id <- seq.int(1, nrow(data_resp_loc))
data_resp_loc$Score <- as.numeric(data_resp_loc$Score) - 1

#data_resp_loc$cond <- plyr::mapvalues(data_resp_loc$cond, 
                  #                from = c("0_0", "0_1", "1_0", "1_1"),
                        #          to = c("Short_Short", "Short_Long", "Long_Short", "Long_Long"))

data_resp_loc_old <- cbind(data_resp_loc)


/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Need help getting started? Try the cookbook for R:
http://www.cookbook-r.com/Graphs/

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following object is masked from ‘package:reshape’:

    rename


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/pyth

In [13]:
%%R -o data_resp_avs

data_resp_loc <- cbind(data_resp_loc_old)
        
data_resp_avs <- data_resp_loc %>%
                  filter((gazeDist > 0)) %>% # & (Trial_End >= 1523639984)) %>%
                  group_by(trial_id, which_array) %>%
                  mutate(arr1 = (which_array==1)*1) %>%
                  mutate(arr2 = (which_array==2)*1) %>%
                  mutate(belowX1 = mean(belowX) * arr1) %>%
                  mutate(belowX2 = mean(belowX) * arr2)%>%
                 mutate(pathLength1 = mean(pathLength) * arr1) %>%
                  mutate(pathLength2 = mean(pathLength) * arr2)%>%
                  mutate(medFix1 = mean(medFix) * arr1) %>%
                  mutate(medFix2 = mean(medFix) * arr2) %>%
                  mutate(pctArea1 = mean(pctArea) * arr1) %>%
                  mutate(pctArea2 = mean(pctArea) * arr2) %>%
                  #mutate(within_x1 = 2/(1 + exp(-1 -0.25*belowX)) - 1) %>%
                 #mutate(within_x2 = 2/(1 + exp(-1 -0.25*belowX)) - 1) %>%
                 # mutate(within_x1 = arr1* mean(within_x1)) %>%
                 # mutate(within_x2 = arr2*mean(within_x2)) %>%
                  mutate(within_x1 = mean(pInGaze) * arr1) %>%
                  mutate(within_x2 = mean(pInGaze)*arr2) %>%

                  mutate(pctArea1 = mean(pctArea) * arr1) %>%
                  mutate(pctArea2 = mean(pctArea) * arr2)%>%
                  mutate(totArea1 = mean(totArea) * arr1) %>%
                  mutate(totArea2 = mean(totArea) * arr2)%>%
                  top_n(n=1, wt=id)%>%
                  group_by(pid, Trial, Time1, Time2) %>%
                  mutate(pctArea1 = sum(pctArea1)) %>%
                  mutate(pctArea2 = sum(pctArea2)) %>%
                  mutate(belowX1 = sum(belowX1)) %>%
                  mutate(belowX2 = sum(belowX2)) %>%
                  mutate(within_x1 = sum(within_x1)) %>%
                  mutate(within_x2=sum(within_x2)) %>%
                  mutate(pathLength1 = sum(pathLength1)) %>%
                  mutate(pathLength2 = sum(pathLength2))%>%
                  mutate(medFix1 = sum(medFix1)) %>%
                  mutate(medFix2 = sum(medFix2))%>%
                 top_n(n=1, wt=id) %>%
                mutate(within_x1_pct = within_x1 / Dots_Shown1) %>%
                mutate(within_x2_pct = within_x2 / Dots_Shown2) %>% 
                mutate(within_x_pct_ratio = (within_x1 - within_x2)/((within_x1**2 + within_x2**2)**0.5)) %>%
               mutate(cond=paste(round(Time1),round(Time2),sep="_"))  %>%
                mutate(log_Dots_Diff =  log(Dots_Shown1) - log(Dots_Shown2)) %>%
                mutate(Dots_Ratio = (Dots_Shown1 - Dots_Shown2)/(Dots_Shown1**2 + Dots_Shown2**2)**0.5) %>%
                mutate(Abs_Dots_Ratio = abs(Dots_Ratio))  %>%
                mutate(Dots_Counted = Dots_Counted - 1) %>%
                mutate(Score = Score - 1)



In [14]:
data_resp = pandas2ri.ri2py(data_resp_avs)

In [18]:

def remove_n(ls,n):
    ret = []
    for l in ls:
        l_rem = l[:n]
        ret.append(l_rem)
    return ret



subj_group = data_resp.groupby("pid")
dots_shown1,dots_shown2,dots_counted,pct_area1,pct_area2,seen1,seen2=[],[],[],[],[],[],[]
for i,subj in subj_group:
    dots_counted.append(list(subj["Dots_Counted"]))
    dots_shown1.append(list(subj["Dots_Shown1"]))
    pct_area1.append(list(subj["pctArea1"]))
    seen1.append(list(subj["within_x1"]))
    #seen1.append(list(subj["within_x1_tot"]))
    dots_shown2.append(list(subj["Dots_Shown2"]))
    pct_area2.append(list(subj["pctArea2"]))
    #seen2.append(list(subj["within_x2_tot"]))
    seen2.append(list(subj["within_x2"]))
    
ns_1 = [len(n) for n in seen1]
min_n1 = min(ns_1)
ns_2 = [len(n) for n in seen2]
min_n2 = min(ns_2)
min_n = min(min_n1, min_n2)

dots_counted = remove_n(dots_counted, min_n)
dots_shown1 = remove_n(dots_shown1, min_n)
pct_area1 = remove_n(pct_area1, min_n)
seen1 = remove_n(seen1, min_n)
dots_shown2 = remove_n(dots_shown2, min_n)
pct_area2 = remove_n(pct_area2, min_n)
seen2 = remove_n(seen2, min_n)

dots_counted=np.array(dots_counted)
dots_shown1=np.array(dots_shown1)
pct_area1=np.array(pct_area1)
seen1=np.array(seen1)
unseen1 = dots_shown1 - seen1
dots_shown2=np.array(dots_shown2)
pct_area2=np.array(pct_area2)
seen2=np.array(seen2)
unseen2 = dots_shown2 - seen2
dims = (len(seen1),len(seen1[0]))

with pm.Model() as mod:
    weber = pm.HalfNormal("weber",sd=1,testval=0.25, shape=(len(seen1),1))
    
        
    #guess_mean1 = pm.Normal("guess1", mu=dots_shown1, sd=weber*dots_shown1)
    #guess_mean2 = pm.Normal("guess2", mu=dots_shown2, sd=weber*dots_shown2)

    
   # guess_mean1 = dots_shown1
   # guess_mean2 = dots_shown2
    guess_mean1 = seen1
    guess_mean2 = seen2
    
    

    
    ps = (guess_mean2 - guess_mean1) /  (weber * ((guess_mean2**2 + guess_mean1**2)**0.5))

    ps = 0.5 * (1. + tt.erf(ps/(2.**0.5))) 
        
    #rand_prob = pm.Exponential('rand_prob', 10)
    #noise = rand_prob * np.ones(dims)
    #rand_flips = pm.Bernoulli('rand_flip', noise, shape=dims)
    #noisy_ps = ps + (noise - ps) * rand_flips
    
    noisy_ps = ps
    
    
    likelihood = pm.Bernoulli('likelihood', noisy_ps,  observed=dots_counted, shape=dims)

    trace = pm.sample(5000, tune=200, cores=2,chains=1, njobs=2)

summary = pm.summary(trace)
print(summary)
    
    


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/usr/local/lib/python3.5/dist-packages/pymc3/model.py:384: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(var.dtype, float):
ERROR (theano.gof.opt): Optimization failure due to: local_grad_log_erfc_neg
ERROR (theano.gof.opt): node: Elemwise{true_div,no_inplace}(Elemwise{mul,no_inplace}.0, Elemwise{erfc,no_inplace}.0)
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/theano/gof/opt.py", line 2019, in process_node
    replacements = lopt.transform(node)
  File "/usr/local/lib/python3.5/dist-packages/theano/tensor/opt.py", line 6780, in local_grad_log_erfc_neg
    if not exp.owner.inputs[0].owner:
AttributeError: 'NoneType' object has no attribute 'owner'

Sequential sampl

ValueError: Bad initial energy: inf. The model might be misspecified.

In [16]:
summary_mean = summary["mean"]

vals = {}
rownames = list(summary.index)
for i in range(len(rownames)):
    rowname = rownames[i]
    if "rand" not in rowname:
        rowname = rowname[:rowname.find("__")]
        val = [summary_mean[i] for _ in range(len(range(ns_1[i%len(ns_1)])))]

        if rowname not in vals:
            vals[rowname] = copy.copy(val)
        else:
            vals[rowname] += copy.copy(val)
        

summary_vals = {}
for val in vals:
    summary_vals[val] = vals[val]
    
summary_vals = pd.DataFrame(summary_vals)
for i in data_resp:
    summary_vals[i] = data_resp[i]
pandas2ri.py2ri(summary_vals)



weber,X,pid,...,log_Dots_Diff,Dots_Ratio,Abs_Dots_Ratio
1.731571,109,0.000000,...,0.095310,0.067267,0.067267
1.731571,231,0.000000,...,0.349674,0.241182,0.241182
1.731571,331,0.000000,...,-0.076961,-0.054353,0.054353
1.731571,441,0.000000,...,-0.018019,-0.012740,0.012740
...,...,...,...,...,...,...
1.503488,155119,23.000000,...,0.095310,0.067267,0.067267
1.503488,155192,23.000000,...,0.247836,0.173045,0.173045


In [17]:
%%R -i summary_vals summary 

summary_vals <- summary_vals %>%
                mutate(ratio=(Dots_Shown1-Dots_Shown2)/(weber*(Dots_Shown1**2 + Dots_Shown2**2)**0.5))

mean_reg_vals <- summary_vals %>%
                group_by(pid, Subject, Time1, Time2) %>%
                mutate(weber = mean(weber)) %>%
                group_by(pid, Subject) %>%
                top_n(n=1,wt=id)



mean_reg_vals <- mean_reg_vals[order(mean_reg_vals$Subject),]

for (i in mean_reg_vals$weber) {
    cat(i)
    cat("\n")
}



1.243513
1.731571
1.891921
1.553395
1.710064
1.58816
1.7846
1.761709
1.783629
1.565477
1.613548
1.503488
1.582992
2.024344
1.969623
1.471056
1.64921
1.461554
1.493016
1.668488
1.844721
1.393017
1.689737
1.615244
